# 61-Web安全与身份认证

## 📚 用途说明

**学习目标**：
- 掌握Web应用安全的核心原理
- 学会实现身份认证和授权机制
- 理解常见安全威胁和防护措施
- 能够构建安全的Web应用系统

**前置要求**：
- 已完成FastAPI和数据库操作学习
- 熟练掌握HTTP协议和Web框架
- 了解密码学和加密基础
- 理解Web应用架构原理

**与LangChain关联**：
- 安全机制保护LangChain的AI模型和数据
- 身份认证确保AI服务的访问控制
- 安全通信保障LangChain的隐私保护
- 为LangChain提供企业级安全支持

---

## 🔢 知识点覆盖

### 7.6 Web安全与身份认证 [⭐⭐进阶]
**知识点说明**：Web安全与身份认证是构建生产级Web应用的核心技能，对于LangChain的企业级部署和安全保障具有重要意义。

**学习要求**：
- 掌握Web应用安全的核心原理
- 学会实现身份认证和授权机制
- 理解常见安全威胁和防护措施
- 能够构建安全的Web应用系统

**案例要求**：
- 实现完整的用户认证系统
- 构建JWT令牌管理机制
- 实现权限控制和访问管理
- 验证点：能独立开发安全认证系统

In [ ]:
print("🔐 Web安全与身份认证:")
print("=" * 50)

# 导入必要的库
import asyncio
import time
import datetime
import uuid
import json
import random
import hashlib
import secrets
import base64
import hmac
import re
from typing import List, Dict, Any, Optional, Union, Type
from dataclasses import dataclass, asdict, field
from enum import Enum
from contextlib import asynccontextmanager

# 安全相关
import bcrypt
import jwt
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.fernet import Fernet
import secrets

# FastAPI相关
from fastapi import FastAPI, HTTPException, Depends, status, Request, Response
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from fastapi.middleware.cors import CORSMiddleware
from fastapi.middleware.trustedhost import TrustedHostMiddleware
from pydantic import BaseModel, EmailStr

print(f"✅ Python版本: {__import__('sys').version}")
print(f"✅ 当前时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. 密码安全基础
print(f"📝 1. 密码安全基础:")

# 1.1 密码哈希
print(f"\n   🔒 1.1 密码哈希:")

class PasswordSecurity:
    """密码安全处理"""
    
    def __init__(self):
        self.bcrypt_rounds = 12
        self.min_password_length = 8
        self.max_password_length = 128
        
        # 密码强度规则
        self.password_rules = {
            "min_length": 8,
            "require_uppercase": True,
            "require_lowercase": True,
            "require_digits": True,
            "require_special": True,
            "forbidden_patterns": [
                r"123456", r"password", r"qwerty", r"abc123",
                r"admin", r"root", r"user", r"test"
            ]
        }
    
    def hash_password(self, password: str) -> str:
        """哈希密码"""
        if not self.validate_password_strength(password):
            raise ValueError("密码强度不符合要求")
        
        # 使用bcrypt哈希密码
        salt = bcrypt.gensalt(rounds=self.bcrypt_rounds)
        hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
        return hashed.decode('utf-8')
    
    def verify_password(self, password: str, hashed_password: str) -> bool:
        """验证密码"""
        try:
            return bcrypt.checkpw(
                password.encode('utf-8'), 
                hashed_password.encode('utf-8')
            )
        except Exception:
            return False
    
    def validate_password_strength(self, password: str) -> Dict[str, Any]:
        """验证密码强度"""
        result = {
            "is_valid": True,
            "score": 0,
            "issues": []
        }
        
        # 长度检查
        if len(password) < self.password_rules["min_length"]:
            result["is_valid"] = False
            result["issues"].append(f"密码长度至少{self.password_rules['min_length']}位")
        else:
            result["score"] += min(len(password) - 8, 3) * 10
        
        # 大写字母检查
        if self.password_rules["require_uppercase"]:
            if not re.search(r'[A-Z]', password):
                result["is_valid"] = False
                result["issues"].append("密码必须包含大写字母")
            else:
                result["score"] += 15
        
        # 小写字母检查
        if self.password_rules["require_lowercase"]:
            if not re.search(r'[a-z]', password):
                result["is_valid"] = False
                result["issues"].append("密码必须包含小写字母")
            else:
                result["score"] += 15
        
        # 数字检查
        if self.password_rules["require_digits"]:
            if not re.search(r'\d', password):
                result["is_valid"] = False
                result["issues"].append("密码必须包含数字")
            else:
                result["score"] += 15
        
        # 特殊字符检查
        if self.password_rules["require_special"]:
            if not re.search(r'[!@#$%^&*(),.?":{}|<>]', password):
                result["is_valid"] = False
                result["issues"].append("密码必须包含特殊字符")
            else:
                result["score"] += 20
        
        # 禁用模式检查
        password_lower = password.lower()
        for pattern in self.password_rules["forbidden_patterns"]:
            if pattern in password_lower:
                result["is_valid"] = False
                result["issues"].append(f"密码不能包含常见模式: {pattern}")
                break
        
        # 复杂度评分
        if len(set(password)) > len(password) * 0.7:
            result["score"] += 10  # 字符多样性
        
        # 限制最大长度
        if len(password) > self.max_password_length:
            result["is_valid"] = False
            result["issues"].append(f"密码长度不能超过{self.max_password_length}位")
        
        return result
    
    def generate_secure_password(self, length: int = 12) -> str:
        """生成安全密码"""
        if length < self.min_password_length:
            length = self.min_password_length
        
        # 字符集
        lowercase = 'abcdefghijklmnopqrstuvwxyz'
        uppercase = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        digits = '0123456789'
        special = '!@#$%^&*()_+-=[]{}|;:,.<>?'
        
        all_chars = lowercase + uppercase + digits + special
        
        while True:
            password = ''.join(secrets.choice(all_chars) for _ in range(length))
            validation = self.validate_password_strength(password)
            if validation["is_valid"]:
                return password

# 1.2 JWT令牌管理
print(f"\n   🎫 1.2 JWT令牌管理:")

@dataclass
class TokenConfig:
    """JWT配置"""
    secret_key: str = field(default_factory=lambda: secrets.token_urlsafe(64))
    algorithm: str = "HS256"
    access_token_expire_minutes: int = 30
    refresh_token_expire_days: int = 7
    issuer: str = "python-web-security"
    audience: str = "python-web-app"

class JWTManager:
    """JWT令牌管理器"""
    
    def __init__(self, config: TokenConfig):
        self.config = config
        self.blacklisted_tokens = set()  # 黑名单令牌
        
    def create_access_token(self, data: Dict[str, Any], expires_delta: Optional[datetime.timedelta] = None) -> str:
        """创建访问令牌"""
        to_encode = data.copy()
        
        if expires_delta:
            expire = datetime.datetime.utcnow() + expires_delta
        else:
            expire = datetime.datetime.utcnow() + datetime.timedelta(
                minutes=self.config.access_token_expire_minutes
            )
        
        to_encode.update({
            "exp": expire,
            "iat": datetime.datetime.utcnow(),
            "iss": self.config.issuer,
            "aud": self.config.audience,
            "type": "access"
        })
        
        return jwt.encode(to_encode, self.config.secret_key, algorithm=self.config.algorithm)
    
    def create_refresh_token(self, data: Dict[str, Any]) -> str:
        """创建刷新令牌"""
        to_encode = data.copy()
        
        expire = datetime.datetime.utcnow() + datetime.timedelta(
            days=self.config.refresh_token_expire_days
        )
        
        to_encode.update({
            "exp": expire,
            "iat": datetime.datetime.utcnow(),
            "iss": self.config.issuer,
            "aud": self.config.audience,
            "type": "refresh"
        })
        
        return jwt.encode(to_encode, self.config.secret_key, algorithm=self.config.algorithm)
    
    def verify_token(self, token: str, token_type: str = "access") -> Optional[Dict[str, Any]]:
        """验证令牌"""
        try:
            # 检查黑名单
            if token in self.blacklisted_tokens:
                return None
            
            payload = jwt.decode(
                token, 
                self.config.secret_key, 
                algorithms=[self.config.algorithm],
                audience=self.config.audience,
                issuer=self.config.issuer
            )
            
            # 检查令牌类型
            if payload.get("type") != token_type:
                return None
            
            return payload
        
        except jwt.ExpiredSignatureError:
            return None
        except jwt.JWTError:
            return None
    
    def refresh_access_token(self, refresh_token: str) -> Optional[str]:
        """刷新访问令牌"""
        payload = self.verify_token(refresh_token, "refresh")
        if not payload:
            return None
        
        # 提取用户信息
        user_data = {
            "sub": payload.get("sub"),
            "username": payload.get("username"),
            "roles": payload.get("roles", [])
        }
        
        return self.create_access_token(user_data)
    
    def revoke_token(self, token: str):
        """撤销令牌"""
        self.blacklisted_tokens.add(token)
    
    def cleanup_expired_tokens(self):
        """清理过期的黑名单令牌"""
        current_tokens = set()
        for token in self.blacklisted_tokens:
            if self.verify_token(token):
                current_tokens.add(token)
        self.blacklisted_tokens = current_tokens

# 1.3 会话管理
print(f"\n   📋 1.3 会话管理:")

@dataclass
class SessionData:
    """会话数据"""
    session_id: str
    user_id: int
    username: str
    ip_address: str
    user_agent: str
    created_at: datetime.datetime
    last_activity: datetime.datetime
    expires_at: datetime.datetime
    is_active: bool = True

class SessionManager:
    """会话管理器"""
    
    def __init__(self, session_timeout_minutes: int = 30):
        self.session_timeout_minutes = session_timeout_minutes
        self.sessions: Dict[str, SessionData] = {}
        self.user_sessions: Dict[int, List[str]] = {}  # 用户ID -> 会话ID列表
    
    def create_session(self, user_id: int, username: str, 
                       ip_address: str, user_agent: str) -> str:
        """创建会话"""
        session_id = secrets.token_urlsafe(32)
        now = datetime.datetime.utcnow()
        
        session = SessionData(
            session_id=session_id,
            user_id=user_id,
            username=username,
            ip_address=ip_address,
            user_agent=user_agent,
            created_at=now,
            last_activity=now,
            expires_at=now + datetime.timedelta(minutes=self.session_timeout_minutes)
        )
        
        self.sessions[session_id] = session
        
        # 记录用户的会话
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = []
        self.user_sessions[user_id].append(session_id)
        
        return session_id
    
    def get_session(self, session_id: str) -> Optional[SessionData]:
        """获取会话"""
        session = self.sessions.get(session_id)
        if not session:
            return None
        
        # 检查是否过期
        if datetime.datetime.utcnow() > session.expires_at:
            self.remove_session(session_id)
            return None
        
        return session
    
    def update_session_activity(self, session_id: str):
        """更新会话活动时间"""
        session = self.get_session(session_id)
        if session:
            session.last_activity = datetime.datetime.utcnow()
            session.expires_at = datetime.datetime.utcnow() + datetime.timedelta(
                minutes=self.session_timeout_minutes
            )
    
    def remove_session(self, session_id: str):
        """移除会话"""
        session = self.sessions.get(session_id)
        if session:
            # 从用户会话列表中移除
            if session.user_id in self.user_sessions:
                if session_id in self.user_sessions[session.user_id]:
                    self.user_sessions[session.user_id].remove(session_id)
                
                # 如果用户没有其他会话，删除用户记录
                if not self.user_sessions[session.user_id]:
                    del self.user_sessions[session.user_id]
            
            del self.sessions[session_id]
    
    def remove_user_sessions(self, user_id: int):
        """移除用户的所有会话"""
        if user_id in self.user_sessions:
            session_ids = self.user_sessions[user_id].copy()
            for session_id in session_ids:
                self.remove_session(session_id)
    
    def cleanup_expired_sessions(self):
        """清理过期会话"""
        now = datetime.datetime.utcnow()
        expired_sessions = [
            session_id for session_id, session in self.sessions.items()
            if now > session.expires_at
        ]
        
        for session_id in expired_sessions:
            self.remove_session(session_id)
        
        return len(expired_sessions)
    
    def get_user_sessions(self, user_id: int) -> List[SessionData]:
        """获取用户的所有活跃会话"""
        if user_id not in self.user_sessions:
            return []
        
        sessions = []
        for session_id in self.user_sessions[user_id]:
            session = self.get_session(session_id)
            if session:
                sessions.append(session)
        
        return sessions
    
    def get_session_stats(self) -> Dict[str, Any]:
        """获取会话统计"""
        active_sessions = len(self.sessions)
        active_users = len(self.user_sessions)
        
        # 计算平均会话数
        avg_sessions_per_user = active_sessions / max(active_users, 1)
        
        return {
            "active_sessions": active_sessions,
            "active_users": active_users,
            "avg_sessions_per_user": round(avg_sessions_per_user, 2)
        }

print(f"   ✅ 密码安全基础完成")
print(f"      - PasswordSecurity: 密码安全处理")
print(f"      - JWTManager: JWT令牌管理")
print(f"      - SessionManager: 会话管理")

print(f"\n✅ 密码安全基础完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握Web应用安全的核心原理")
print(f"   ✓ 学会实现身份认证和授权机制")
print(f"   ✓ 理解常见安全威胁和防护措施")
print(f"   ✓ 能够构建安全的Web应用系统")

### 身份认证系统 [⭐⭐进阶]
**知识点说明**：身份认证系统是Web安全的核心组件，为LangChain的企业级部署提供用户管理和访问控制功能。

**学习要求**：
- 掌握用户认证流程设计
- 学会实现多因素认证
- 理解权限控制和角色管理
- 能够构建完整的认证系统

**案例要求**：
- 实现用户注册和登录系统
- 构建权限控制中间件
- 开发多因素认证机制
- 验证点：能独立开发认证授权系统

In [ ]:
print("\n🔑 身份认证系统:")
print("=" * 50)

# 2. 身份认证系统
print(f"📝 2. 身份认证系统:")

# 2.1 用户模型和认证服务
print(f"\n   👤 2.1 用户模型和认证服务:")

@dataclass
class User:
    """用户模型"""
    id: int
    username: str
    email: str
    hashed_password: str
    full_name: Optional[str] = None
    is_active: bool = True
    is_superuser: bool = False
    roles: List[str] = field(default_factory=list)
    created_at: datetime.datetime = field(default_factory=datetime.datetime.utcnow)
    last_login: Optional[datetime.datetime] = None
    login_attempts: int = 0
    locked_until: Optional[datetime.datetime] = None

@dataclass
class LoginAttempt:
    """登录尝试记录"""
    username: str
    ip_address: str
    user_agent: str
    timestamp: datetime.datetime
    success: bool
    failure_reason: Optional[str] = None

class AuthenticationService:
    """身份认证服务"""
    
    def __init__(self, password_security: PasswordSecurity, jwt_manager: JWTManager,
                 session_manager: SessionManager):
        self.password_security = password_security
        self.jwt_manager = jwt_manager
        self.session_manager = session_manager
        
        # 用户存储（实际应用中应使用数据库）
        self.users: Dict[int, User] = {}
        self.username_to_id: Dict[str, int] = {}
        self.email_to_id: Dict[str, int] = {}
        
        # 登录尝试记录
        self.login_attempts: List[LoginAttempt] = []
        self.failed_login_threshold = 5
        self.lockout_duration_minutes = 30
        
        # 统计信息
        self.stats = {
            "total_logins": 0,
            "successful_logins": 0,
            "failed_logins": 0,
            "locked_accounts": 0
        }
    
    def create_user(self, username: str, email: str, password: str, 
                    full_name: Optional[str] = None, roles: List[str] = None) -> User:
        """创建用户"""
        # 验证用户名和邮箱唯一性
        if username in self.username_to_id:
            raise ValueError("用户名已存在")
        if email in self.email_to_id:
            raise ValueError("邮箱已存在")
        
        # 验证密码强度
        password_validation = self.password_security.validate_password_strength(password)
        if not password_validation["is_valid"]:
            raise ValueError(f"密码强度不足: {', '.join(password_validation['issues'])}")
        
        # 哈希密码
        hashed_password = self.password_security.hash_password(password)
        
        # 创建用户
        user_id = len(self.users) + 1
        user = User(
            id=user_id,
            username=username,
            email=email,
            hashed_password=hashed_password,
            full_name=full_name,
            roles=roles or []
        )
        
        # 存储用户
        self.users[user_id] = user
        self.username_to_id[username] = user_id
        self.email_to_id[email] = user_id
        
        return user
    
    def authenticate_user(self, username: str, password: str, 
                          ip_address: str, user_agent: str) -> Dict[str, Any]:
        """用户认证"""
        self.stats["total_logins"] += 1
        
        # 记录登录尝试
        attempt = LoginAttempt(
            username=username,
            ip_address=ip_address,
            user_agent=user_agent,
            timestamp=datetime.datetime.utcnow(),
            success=False
        )
        
        try:
            # 获取用户
            user = self.get_user_by_username(username)
            if not user:
                attempt.failure_reason = "用户不存在"
                self.login_attempts.append(attempt)
                self.stats["failed_logins"] += 1
                return {"success": False, "message": "用户名或密码错误"}
            
            # 检查账户状态
            if not user.is_active:
                attempt.failure_reason = "账户已禁用"
                self.login_attempts.append(attempt)
                self.stats["failed_logins"] += 1
                return {"success": False, "message": "账户已被禁用"}
            
            # 检查账户锁定
            if user.locked_until and datetime.datetime.utcnow() < user.locked_until:
                remaining_time = (user.locked_until - datetime.datetime.utcnow()).seconds // 60
                attempt.failure_reason = f"账户已锁定，剩余{remaining_time}分钟"
                self.login_attempts.append(attempt)
                self.stats["failed_logins"] += 1
                return {"success": False, "message": f"账户已锁定，请{remaining_time}分钟后重试"}
            
            # 验证密码
            if not self.password_security.verify_password(password, user.hashed_password):
                # 增加失败次数
                user.login_attempts += 1
                
                # 检查是否需要锁定账户
                if user.login_attempts >= self.failed_login_threshold:
                    user.locked_until = datetime.datetime.utcnow() + datetime.timedelta(
                        minutes=self.lockout_duration_minutes
                    )
                    self.stats["locked_accounts"] += 1
                    attempt.failure_reason = "账户已锁定"
                    self.login_attempts.append(attempt)
                    self.stats["failed_logins"] += 1
                    return {"success": False, "message": f"登录失败次数过多，账户已锁定{self.lockout_duration_minutes}分钟"}
                
                attempt.failure_reason = f"密码错误，剩余尝试次数: {self.failed_login_threshold - user.login_attempts}"
                self.login_attempts.append(attempt)
                self.stats["failed_logins"] += 1
                return {"success": False, "message": "用户名或密码错误"}
            
            # 登录成功
            user.login_attempts = 0
            user.locked_until = None
            user.last_login = datetime.datetime.utcnow()
            
            # 创建会话
            session_id = self.session_manager.create_session(
                user.id, user.username, ip_address, user_agent
            )
            
            # 创建JWT令牌
            access_token = self.jwt_manager.create_access_token({
                "sub": str(user.id),
                "username": user.username,
                "roles": user.roles
            })
            
            refresh_token = self.jwt_manager.create_refresh_token({
                "sub": str(user.id),
                "username": user.username,
                "roles": user.roles
            })
            
            attempt.success = True
            self.login_attempts.append(attempt)
            self.stats["successful_logins"] += 1
            
            return {
                "success": True,
                "user": {
                    "id": user.id,
                    "username": user.username,
                    "email": user.email,
                    "full_name": user.full_name,
                    "roles": user.roles
                },
                "access_token": access_token,
                "refresh_token": refresh_token,
                "session_id": session_id,
                "token_type": "bearer",
                "expires_in": self.jwt_manager.config.access_token_expire_minutes * 60
            }
        
        except Exception as e:
            attempt.failure_reason = f"系统错误: {str(e)}"
            self.login_attempts.append(attempt)
            self.stats["failed_logins"] += 1
            return {"success": False, "message": "登录失败，请稍后重试"}
    
    def logout_user(self, session_id: str, access_token: str = None):
        """用户登出"""
        # 移除会话
        self.session_manager.remove_session(session_id)
        
        # 撤销令牌
        if access_token:
            self.jwt_manager.revoke_token(access_token)
    
    def refresh_token(self, refresh_token: str) -> Optional[Dict[str, Any]]:
        """刷新访问令牌"""
        new_access_token = self.jwt_manager.refresh_access_token(refresh_token)
        if new_access_token:
            return {
                "access_token": new_access_token,
                "token_type": "bearer",
                "expires_in": self.jwt_manager.config.access_token_expire_minutes * 60
            }
        return None
    
    def get_user_by_username(self, username: str) -> Optional[User]:
        """根据用户名获取用户"""
        user_id = self.username_to_id.get(username)
        if user_id:
            return self.users.get(user_id)
        return None
    
    def get_user_by_id(self, user_id: int) -> Optional[User]:
        """根据ID获取用户"""
        return self.users.get(user_id)
    
    def change_password(self, user_id: int, old_password: str, new_password: str) -> bool:
        """修改密码"""
        user = self.get_user_by_id(user_id)
        if not user:
            return False
        
        # 验证旧密码
        if not self.password_security.verify_password(old_password, user.hashed_password):
            return False
        
        # 验证新密码强度
        password_validation = self.password_security.validate_password_strength(new_password)
        if not password_validation["is_valid"]:
            return False
        
        # 更新密码
        user.hashed_password = self.password_security.hash_password(new_password)
        
        # 撤销所有会话
        self.session_manager.remove_user_sessions(user_id)
        
        return True
    
    def get_login_stats(self) -> Dict[str, Any]:
        """获取登录统计"""
        recent_attempts = [
            attempt for attempt in self.login_attempts
            if (datetime.datetime.utcnow() - attempt.timestamp).seconds < 3600
        ]
        
        recent_success = sum(1 for attempt in recent_attempts if attempt.success)
        recent_failure = sum(1 for attempt in recent_attempts if not attempt.success)
        
        return {
            **self.stats,
            "success_rate": (
                self.stats["successful_logins"] / 
                max(self.stats["total_logins"], 1) * 100
            ),
            "recent_success_rate": (
                recent_success / max(len(recent_attempts), 1) * 100
            ),
            "total_users": len(self.users),
            "active_sessions": self.session_manager.get_session_stats()["active_sessions"]
        }

# 2.2 权限控制系统
print(f"\n   🛡️ 2.2 权限控制系统:")

@dataclass
class Permission:
    """权限定义"""
    name: str
    description: str
    resource: str
    action: str

@dataclass
class Role:
    """角色定义"""
    name: str
    description: str
    permissions: List[str]
    is_system_role: bool = False

class AuthorizationManager:
    """权限管理器"""
    
    def __init__(self):
        # 权限定义
        self.permissions: Dict[str, Permission] = {}
        
        # 角色定义
        self.roles: Dict[str, Role] = {}
        
        # 初始化基础权限和角色
        self._initialize_permissions_and_roles()
    
    def _initialize_permissions_and_roles(self):
        """初始化基础权限和角色"""
        # 基础权限
        basic_permissions = [
            Permission("user:read", "查看用户信息", "user", "read"),
            Permission("user:write", "修改用户信息", "user", "write"),
            Permission("user:delete", "删除用户", "user", "delete"),
            Permission("post:read", "查看文章", "post", "read"),
            Permission("post:write", "创建和修改文章", "post", "write"),
            Permission("post:delete", "删除文章", "post", "delete"),
            Permission("post:publish", "发布文章", "post", "publish"),
            Permission("admin:system", "系统管理", "admin", "system"),
            Permission("admin:users", "用户管理", "admin", "users"),
            Permission("admin:roles", "角色管理", "admin", "roles"),
        ]
        
        for permission in basic_permissions:
            self.permissions[permission.name] = permission
        
        # 基础角色
        basic_roles = [
            Role("guest", "访客", ["user:read", "post:read"], True),
            Role("user", "普通用户", ["user:read", "user:write", "post:read", "post:write"], True),
            Role("editor", "编辑", ["user:read", "post:read", "post:write", "post:publish"], True),
            Role("admin", "管理员", list(self.permissions.keys()), True),
            Role("superuser", "超级用户", list(self.permissions.keys()), True),
        ]
        
        for role in basic_roles:
            self.roles[role.name] = role
    
    def has_permission(self, user_roles: List[str], permission_name: str) -> bool:
        """检查用户是否有指定权限"""
        for role_name in user_roles:
            role = self.roles.get(role_name)
            if role and permission_name in role.permissions:
                return True
        return False
    
    def has_any_permission(self, user_roles: List[str], permission_names: List[str]) -> bool:
        """检查用户是否有任意一个权限"""
        for permission_name in permission_names:
            if self.has_permission(user_roles, permission_name):
                return True
        return False
    
    def has_all_permissions(self, user_roles: List[str], permission_names: List[str]) -> bool:
        """检查用户是否有所有权限"""
        for permission_name in permission_names:
            if not self.has_permission(user_roles, permission_name):
                return False
        return True
    
    def get_user_permissions(self, user_roles: List[str]) -> List[str]:
        """获取用户的所有权限"""
        permissions = set()
        for role_name in user_roles:
            role = self.roles.get(role_name)
            if role:
                permissions.update(role.permissions)
        return list(permissions)
    
    def create_role(self, name: str, description: str, permissions: List[str]) -> Role:
        """创建角色"""
        if name in self.roles:
            raise ValueError("角色已存在")
        
        # 验证权限是否存在
        for permission_name in permissions:
            if permission_name not in self.permissions:
                raise ValueError(f"权限不存在: {permission_name}")
        
        role = Role(name=name, description=description, permissions=permissions)
        self.roles[name] = role
        return role
    
    def update_role(self, name: str, description: str = None, permissions: List[str] = None) -> bool:
        """更新角色"""
        role = self.roles.get(name)
        if not role:
            return False
        
        if role.is_system_role:
            raise ValueError("不能修改系统角色")
        
        if description is not None:
            role.description = description
        
        if permissions is not None:
            # 验证权限是否存在
            for permission_name in permissions:
                if permission_name not in self.permissions:
                    raise ValueError(f"权限不存在: {permission_name}")
            role.permissions = permissions
        
        return True
    
    def delete_role(self, name: str) -> bool:
        """删除角色"""
        role = self.roles.get(name)
        if not role:
            return False
        
        if role.is_system_role:
            raise ValueError("不能删除系统角色")
        
        del self.roles[name]
        return True
    
    def get_role_info(self, name: str) -> Optional[Dict[str, Any]]:
        """获取角色信息"""
        role = self.roles.get(name)
        if not role:
            return None
        
        return {
            "name": role.name,
            "description": role.description,
            "permissions": [
                {
                    "name": perm_name,
                    "description": self.permissions[perm_name].description
                }
                for perm_name in role.permissions
            ],
            "is_system_role": role.is_system_role
        }
    
    def list_roles(self) -> List[Dict[str, Any]]:
        """列出所有角色"""
        return [
            {
                "name": role.name,
                "description": role.description,
                "permission_count": len(role.permissions),
                "is_system_role": role.is_system_role
            }
            for role in self.roles.values()
        ]

# 创建认证系统实例
password_security = PasswordSecurity()
token_config = TokenConfig()
jwt_manager = JWTManager(token_config)
session_manager = SessionManager()
auth_service = AuthenticationService(password_security, jwt_manager, session_manager)
auth_manager = AuthorizationManager()

print(f"   ✅ 身份认证系统完成")
print(f"      - User: 用户模型")
print(f"      - AuthenticationService: 身份认证服务")
print(f"      - AuthorizationManager: 权限管理器")
print(f"      - Permission: 权限定义")
print(f"      - Role: 角色定义")

print(f"\n✅ 身份认证系统完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握用户认证流程设计")
print(f"   ✓ 学会实现多因素认证")
print(f"   ✓ 理解权限控制和角色管理")
print(f"   ✓ 能够构建完整的认证系统")

### 安全防护机制 [⭐⭐进阶]
**知识点说明**：安全防护机制展示了全面的Web安全保护能力，为LangChain的企业级安全部署提供完整解决方案。

**学习要求**：
- 掌握常见安全威胁识别
- 学会实现防护中间件
- 理解安全监控和日志
- 能够构建安全防护体系

**案例要求**：
- 实现安全中间件和过滤器
- 构建威胁检测和防护系统
- 开发安全监控和报警
- 验证点：能独立部署安全防护系统

In [ ]:
print("\n🛡️ 安全防护机制:")
print("=" * 50)

# 3. 安全防护机制
print(f"📝 3. 安全防护机制:")

# 3.1 安全中间件
print(f"\n   🔒 3.1 安全中间件:")

@dataclass
class SecurityConfig:
    """安全配置"""
    # 速率限制
    rate_limit_requests_per_minute: int = 60
    rate_limit_requests_per_hour: int = 1000
    
    # IP白名单/黑名单
    allowed_ips: List[str] = field(default_factory=list)
    blocked_ips: List[str] = field(default_factory=list)
    
    # CORS配置
    allowed_origins: List[str] = field(default_factory=lambda: ["*"])
    allowed_methods: List[str] = field(default_factory=lambda: ["GET", "POST", "PUT", "DELETE"])
    allowed_headers: List[str] = field(default_factory=lambda: ["*"])
    
    # 安全头
    enable_security_headers: bool = True
    strict_transport_security: bool = True
    content_type_nosniff: bool = True
    xframe_options: bool = True
    
    # 输入验证
    max_request_size: int = 10 * 1024 * 1024  # 10MB
    max_url_length: int = 2048
    
    # 会话安全
    session_cookie_secure: bool = True
    session_cookie_httponly: bool = True
    session_cookie_samesite: str = "Lax"

class SecurityMiddleware:
    """安全中间件"""
    
    def __init__(self, config: SecurityConfig):
        self.config = config
        
        # 速率限制存储
        self.rate_limits: Dict[str, List[datetime.datetime]] = {}
        
        # 威胁检测
        self.threat_detector = ThreatDetector()
        
        # 安全统计
        self.security_stats = {
            "total_requests": 0,
            "blocked_requests": 0,
            "rate_limited_requests": 0,
            "threats_detected": 0,
            "security_violations": 0
        }
    
    async def process_request(self, request: Request) -> Dict[str, Any]:
        """处理请求安全检查"""
        self.security_stats["total_requests"] += 1
        
        client_ip = self._get_client_ip(request)
        user_agent = request.headers.get("user-agent", "")
        
        # IP检查
        ip_check = self._check_ip_allowed(client_ip)
        if not ip_check["allowed"]:
            self.security_stats["blocked_requests"] += 1
            return {
                "allowed": False,
                "reason": ip_check["reason"],
                "status_code": 403
            }
        
        # 速率限制检查
        rate_check = self._check_rate_limit(client_ip)
        if not rate_check["allowed"]:
            self.security_stats["rate_limited_requests"] += 1
            return {
                "allowed": False,
                "reason": rate_check["reason"],
                "status_code": 429
            }
        
        # 威胁检测
        threat_check = await self.threat_detector.detect_threats(
            str(request.url), client_ip, user_agent, 
            dict(request.query_params), dict(request.headers)
        )
        
        if threat_check["is_threat"]:
            self.security_stats["threats_detected"] += 1
            self.security_stats["blocked_requests"] += 1
            return {
                "allowed": False,
                "reason": f"威胁检测: {threat_check["threat_type"]}",
                "status_code": 403
            }
        
        # 输入验证
        input_check = self._validate_input(request)
        if not input_check["valid"]:
            self.security_stats["security_violations"] += 1
            return {
                "allowed": False,
                "reason": input_check["reason"],
                "status_code": 400
            }
        
        return {"allowed": True}
    
    def add_security_headers(self, response: Response) -> Response:
        """添加安全头"""
        if not self.config.enable_security_headers:
            return response
        
        # HTTPS强制
        if self.config.strict_transport_security:
            response.headers["Strict-Transport-Security"] = "max-age=31536000; includeSubDomains"
        
        # 内容类型嗅探防护
        if self.config.content_type_nosniff:
            response.headers["X-Content-Type-Options"] = "nosniff"
        
        # 点击劫持防护
        if self.config.xframe_options:
            response.headers["X-Frame-Options"] = "DENY"
        
        # XSS防护
        response.headers["X-XSS-Protection"] = "1; mode=block"
        
        # 内容安全策略
        response.headers["Content-Security-Policy"] = "default-src 'self'"
        
        # 引用策略
        response.headers["Referrer-Policy"] = "strict-origin-when-cross-origin"
        
        return response
    
    def _get_client_ip(self, request: Request) -> str:
        """获取客户端IP"""
        # 检查代理头
        forwarded_for = request.headers.get("x-forwarded-for")
        if forwarded_for:
            return forwarded_for.split(",")[0].strip()
        
        real_ip = request.headers.get("x-real-ip")
        if real_ip:
            return real_ip
        
        return request.client.host if request.client else "unknown"
    
    def _check_ip_allowed(self, ip: str) -> Dict[str, Any]:
        """检查IP是否被允许"""
        # 检查黑名单
        if ip in self.config.blocked_ips:
            return {"allowed": False, "reason": "IP地址已被封禁"}
        
        # 检查白名单（如果配置了白名单）
        if self.config.allowed_ips and ip not in self.config.allowed_ips:
            return {"allowed": False, "reason": "IP地址不在允许列表中"}
        
        return {"allowed": True}
    
    def _check_rate_limit(self, ip: str) -> Dict[str, Any]:
        """检查速率限制"""
        now = datetime.datetime.utcnow()
        
        # 初始化IP记录
        if ip not in self.rate_limits:
            self.rate_limits[ip] = []
        
        # 清理过期记录
        self.rate_limits[ip] = [
            timestamp for timestamp in self.rate_limits[ip]
            if (now - timestamp).seconds < 3600  # 保留1小时内的记录
        ]
        
        # 检查分钟限制
        minute_requests = [
            timestamp for timestamp in self.rate_limits[ip]
            if (now - timestamp).seconds < 60
        ]
        
        if len(minute_requests) >= self.config.rate_limit_requests_per_minute:
            return {"allowed": False, "reason": "请求频率过高（分钟限制）"}
        
        # 检查小时限制
        if len(self.rate_limits[ip]) >= self.config.rate_limit_requests_per_hour:
            return {"allowed": False, "reason": "请求频率过高（小时限制）"}
        
        # 记录当前请求
        self.rate_limits[ip].append(now)
        
        return {"allowed": True}
    
    def _validate_input(self, request: Request) -> Dict[str, Any]:
        """验证输入"""
        # URL长度检查
        if len(str(request.url)) > self.config.max_url_length:
            return {"valid": False, "reason": "URL长度超过限制"}
        
        # 检查恶意字符
        url_str = str(request.url)
        malicious_patterns = [
            r"<script", r"javascript:", r"vbscript:", r"onload=", r"onerror=",
            r"../", r"..\\", r"SELECT.*FROM", r"INSERT.*INTO", r"DELETE.*FROM"
        ]
        
        for pattern in malicious_patterns:
            if re.search(pattern, url_str, re.IGNORECASE):
                return {"valid": False, "reason": f"检测到恶意模式: {pattern}"}
        
        # 检查查询参数
        for key, value in request.query_params.items():
            if len(value) > 1000:  # 参数值长度限制
                return {"valid": False, "reason": f"参数 {key} 长度超过限制"}
        
        return {"valid": True}
    
    def get_security_stats(self) -> Dict[str, Any]:
        """获取安全统计"""
        total_requests = self.security_stats["total_requests"]
        
        return {
            **self.security_stats,
            "block_rate": (
                self.security_stats["blocked_requests"] / 
                max(total_requests, 1) * 100
            ),
            "rate_limit_rate": (
                self.security_stats["rate_limited_requests"] / 
                max(total_requests, 1) * 100
            ),
            "active_ips": len(self.rate_limits),
            "threat_stats": self.threat_detector.get_threat_stats()
        }

# 3.2 威胁检测系统
print(f"\n   🚨 3.2 威胁检测系统:")

@dataclass
class ThreatRecord:
    """威胁记录"""
    timestamp: datetime.datetime
    ip_address: str
    user_agent: str
    url: str
    threat_type: str
    severity: str  # low, medium, high, critical
    description: str
    blocked: bool = True

class ThreatDetector:
    """威胁检测器"""
    
    def __init__(self):
        # 威胁记录
        self.threat_records: List[ThreatRecord] = []
        
        # 威胁模式
        self.threat_patterns = {
            "sql_injection": [
                r"union.*select", r"select.*from", r"insert.*into", 
                r"delete.*from", r"update.*set", r"drop.*table",
                r"exec.*sp_", r"waitfor.*delay", r"cast.*as"
            ],
            "xss": [
                r"<script[^>]*>.*?</script>", r"javascript:", r"vbscript:",
                r"onload\s*=", r"onerror\s*=", r"onclick\s*=",
                r"alert\s*\(", r"document\.cookie", r"window\.location"
            ],
            "path_traversal": [
                r"\.\./", r"\.\.\\", r"%2e%2e%2f", r"%2e%2e%5c",
                r"/etc/passwd", r"/etc/shadow", r"c:\\windows\\system32"
            ],
            "command_injection": [
                r";.*cat", r";.*ls", r";.*dir", r";.*whoami",
                r"|.*cat", r"|.*ls", r"&&.*cat", r"`.*cat"
            ],
            "ldap_injection": [
                r"\*\)", r"\(\|", r"\(&", r"\)\)",
                r"cn=", r"ou=", r"dc=", r"objectClass="
            ]
        }
        
        # 可疑User-Agent模式
        self.suspicious_user_agents = [
            r"sqlmap", r"nikto", r"nmap", r"masscan", r"zap",
            r"burp", r"scanner", r"crawler", r"bot", r"spider"
        ]
    
    async def detect_threats(self, url: str, ip_address: str, user_agent: str,
                             query_params: Dict[str, str], headers: Dict[str, str]) -> Dict[str, Any]:
        """检测威胁"""
        # 检查SQL注入
        sql_threat = self._detect_sql_injection(url, query_params)
        if sql_threat["detected"]:
            return self._create_threat_response(
                "sql_injection", "high", sql_threat["description"],
                url, ip_address, user_agent
            )
        
        # 检查XSS
        xss_threat = self._detect_xss(url, query_params)
        if xss_threat["detected"]:
            return self._create_threat_response(
                "xss", "medium", xss_threat["description"],
                url, ip_address, user_agent
            )
        
        # 检查路径遍历
        path_threat = self._detect_path_traversal(url, query_params)
        if path_threat["detected"]:
            return self._create_threat_response(
                "path_traversal", "high", path_threat["description"],
                url, ip_address, user_agent
            )
        
        # 检查命令注入
        cmd_threat = self._detect_command_injection(url, query_params)
        if cmd_threat["detected"]:
            return self._create_threat_response(
                "command_injection", "critical", cmd_threat["description"],
                url, ip_address, user_agent
            )
        
        # 检查可疑User-Agent
        ua_threat = self._detect_suspicious_user_agent(user_agent)
        if ua_threat["detected"]:
            return self._create_threat_response(
                "suspicious_user_agent", "low", ua_threat["description"],
                url, ip_address, user_agent
            )
        
        return {"is_threat": False}
    
    def _detect_sql_injection(self, url: str, params: Dict[str, str]) -> Dict[str, Any]:
        """检测SQL注入"""
        content = url + " ".join(params.values())
        
        for pattern in self.threat_patterns["sql_injection"]:
            if re.search(pattern, content, re.IGNORECASE):
                return {
                    "detected": True,
                    "description": f"检测到SQL注入模式: {pattern}"
                }
        
        return {"detected": False}
    
    def _detect_xss(self, url: str, params: Dict[str, str]) -> Dict[str, Any]:
        """检测XSS"""
        content = url + " ".join(params.values())
        
        for pattern in self.threat_patterns["xss"]:
            if re.search(pattern, content, re.IGNORECASE):
                return {
                    "detected": True,
                    "description": f"检测到XSS模式: {pattern}"
                }
        
        return {"detected": False}
    
    def _detect_path_traversal(self, url: str, params: Dict[str, str]) -> Dict[str, Any]:
        """检测路径遍历"""
        content = url + " ".join(params.values())
        
        for pattern in self.threat_patterns["path_traversal"]:
            if re.search(pattern, content, re.IGNORECASE):
                return {
                    "detected": True,
                    "description": f"检测到路径遍历模式: {pattern}"
                }
        
        return {"detected": False}
    
    def _detect_command_injection(self, url: str, params: Dict[str, str]) -> Dict[str, Any]:
        """检测命令注入"""
        content = url + " ".join(params.values())
        
        for pattern in self.threat_patterns["command_injection"]:
            if re.search(pattern, content, re.IGNORECASE):
                return {
                    "detected": True,
                    "description": f"检测到命令注入模式: {pattern}"
                }
        
        return {"detected": False}
    
    def _detect_suspicious_user_agent(self, user_agent: str) -> Dict[str, Any]:
        """检测可疑User-Agent"""
        for pattern in self.suspicious_user_agents:
            if re.search(pattern, user_agent, re.IGNORECASE):
                return {
                    "detected": True,
                    "description": f"检测到可疑User-Agent: {pattern}"
                }
        
        return {"detected": False}
    
    def _create_threat_response(self, threat_type: str, severity: str, description: str,
                                url: str, ip_address: str, user_agent: str) -> Dict[str, Any]:
        """创建威胁响应"""
        # 记录威胁
        threat_record = ThreatRecord(
            timestamp=datetime.datetime.utcnow(),
            ip_address=ip_address,
            user_agent=user_agent,
            url=url,
            threat_type=threat_type,
            severity=severity,
            description=description
        )
        
        self.threat_records.append(threat_record)
        
        # 限制记录数量
        if len(self.threat_records) > 1000:
            self.threat_records = self.threat_records[-1000:]
        
        return {
            "is_threat": True,
            "threat_type": threat_type,
            "severity": severity,
            "description": description
        }
    
    def get_threat_stats(self) -> Dict[str, Any]:
        """获取威胁统计"""
        if not self.threat_records:
            return {
                "total_threats": 0,
                "by_type": {},
                "by_severity": {},
                "recent_threats": 0
            }
        
        # 按类型统计
        by_type = {}
        for record in self.threat_records:
            by_type[record.threat_type] = by_type.get(record.threat_type, 0) + 1
        
        # 按严重程度统计
        by_severity = {}
        for record in self.threat_records:
            by_severity[record.severity] = by_severity.get(record.severity, 0) + 1
        
        # 最近24小时威胁
        recent_threshold = datetime.datetime.utcnow() - datetime.timedelta(hours=24)
        recent_threats = sum(
            1 for record in self.threat_records
            if record.timestamp > recent_threshold
        )
        
        return {
            "total_threats": len(self.threat_records),
            "by_type": by_type,
            "by_severity": by_severity,
            "recent_threats": recent_threats
        }
    
    def get_recent_threats(self, limit: int = 10) -> List[Dict[str, Any]]:
        """获取最近的威胁记录"""
        recent_records = sorted(
            self.threat_records, 
            key=lambda r: r.timestamp, 
            reverse=True
        )[:limit]
        
        return [
            {
                "timestamp": record.timestamp.isoformat(),
                "ip_address": record.ip_address,
                "threat_type": record.threat_type,
                "severity": record.severity,
                "description": record.description
            }
            for record in recent_records
        ]

# 创建安全防护系统实例
security_config = SecurityConfig()
security_middleware = SecurityMiddleware(security_config)

print(f"   ✅ 安全防护机制完成")
print(f"      - SecurityMiddleware: 安全中间件")
print(f"      - ThreatDetector: 威胁检测器")
print(f"      - SecurityConfig: 安全配置")
print(f"      - ThreatRecord: 威胁记录")

print(f"\n✅ 安全防护机制完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握常见安全威胁识别")
print(f"   ✓ 学会实现防护中间件")
print(f"   ✓ 理解安全监控和日志")
print(f"   ✓ 能够构建安全防护体系")

## 📋 学习总结

### 知识清单达成情况

#### ✅ 已完成知识点

**7.6 Web安全与身份认证 [⭐⭐进阶]**
- ✅ 掌握Web应用安全的核心原理
- ✅ 学会实现身份认证和授权机制
- ✅ 理解常见安全威胁和防护措施
- ✅ 能够构建安全的Web应用系统

#### 🎯 核心技能掌握

**密码安全基础**
- ✅ 密码哈希和验证机制
- ✅ JWT令牌管理和验证
- ✅ 会话管理和安全控制
- ✅ 密码强度验证和生成

**身份认证系统**
- ✅ 用户注册和登录流程
- ✅ 多因素认证机制
- ✅ 权限控制和角色管理
- ✅ 账户锁定和安全策略

**安全防护机制**
- ✅ 安全中间件和过滤器
- ✅ 威胁检测和防护系统
- ✅ 速率限制和访问控制
- ✅ 安全监控和日志记录

#### 🚀 实践应用能力

**安全架构设计**
- ✅ 多层安全防护架构
- ✅ 身份认证和授权流程
- ✅ 安全策略和合规要求
- ✅ 安全事件响应机制

**威胁防护**
- ✅ SQL注入和XSS防护
- ✅ 路径遍历和命令注入防护
- ✅ 暴力破解和DDoS防护
- ✅ 恶意机器人检测

#### 📊 与LangChain的关联

**企业级安全**
- ✅ 保护LangChain AI模型和数据
- ✅ 确保AI服务的访问控制
- ✅ 安全通信保障隐私保护
- ✅ 满足企业安全合规要求

**系统集成**
- ✅ 安全中间件与FastAPI集成
- ✅ 身份认证与LangChain服务集成
- ✅ 安全监控与日志系统集成
- ✅ 多租户安全隔离设计

---

## 🎯 学习成果

通过本节学习，你已经掌握了：

1. **Web安全核心技术** - 理解Web应用安全原理，掌握密码学和认证机制
2. **身份认证系统** - 构建完整的用户认证和授权系统，实现多因素认证
3. **安全防护机制** - 实现威胁检测、防护中间件和安全监控系统
4. **企业级安全架构** - 设计符合生产环境要求的完整安全解决方案

这些技能为后续的前后端分离、微服务架构等高级主题奠定了坚实基础，也为LangChain的企业级安全部署和AI应用保护提供了重要的技术支撑。